In [1]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'Label_synergy4000.xlsx')

In [2]:
temp1 = []
index_l = []
for index,row in data2.iterrows():
    ct = np.array(data1.loc[row["DrugA"],:]) + np.array(data1.loc[row["DrugB"],:])
    tp = []
    for i in ct:
        if i >= 1:
            tp.append(1)
        else:
            tp.append(0)
    temp1.append(tp)
    index_l.append((row["DrugA"],row["DrugB"]))

In [3]:
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]        
        b=q[i]
        if (a==1)|(b==1):
            tep1=tep1+1
        if (a==1)&(b==1):
            tep2=tep2+1 
    c=float(tep2 / tep1)        
    return c

In [4]:
temp=[]  
for i in temp1:
    tmp=[]   
    a=np.array(i)
    for j in temp1:
        b=np.array(j)
        tp=tanimoto(a,b)
        tmp.append(tp)
    temp.append(tmp)
Tanimoto=pd.DataFrame(temp,columns=index_l,index=index_l)

In [5]:
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1])
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp))
columns = data2.columns

In [6]:
# 删去对应行后的字典
dict_lines = {}
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct])
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y
    ct += 1

In [7]:
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-5:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data

In [8]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
cct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,cct])
    y.pop(count)
    count += 1
    dict_high_lines[str(i)[1:-1]] = y
    tp = []
    for j in columns:
        if str(i)[1:-1] != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[str(i)[1:-1]], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-5:] 
    tp = []
    for j in max_l:
        tp.append(Tanimoto.iloc[list(columns).index(j), :])
    max_5y = list(pd.DataFrame(tp).T.iloc[[list(columns).index(str(i)[1:-1])]].values)[0]
    data = {
        "y": y,
        "max_relevance": max_l,
        "high_5": max_5y
    }
    dict_high_lines[str(i)[1:-1]] = data
    cct += 1

In [9]:
end_score = []
name_dict_high_lines = []
for elem in dict_high_lines:
    end_score.append(dict_high_lines[elem]['high_5'].mean())
    name_dict_high_lines.append(elem)
dict_high_lines_temp = {}
for score in end_score:
    if score >= 0.90:
        dict_high_lines_temp[name_dict_high_lines[end_score.index(score)]] = dict_high_lines[name_dict_high_lines[end_score.index(score)]]
print(len(dict_high_lines_temp))
dict_high_lines = dict_high_lines_temp.copy()

56


In [10]:
def dictW(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    for elem in dict_high_lines:
        r = np.array(dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha,2)))
        dict_W[elem] = W
    return dict_W

#计算MSE
def dictr(dict_W):
# 残差字典 
    # r = max(|A×W - b|)
    dict_r = {}
    dict_r_list = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x
        b = np.array(dict_columns[elem]["y"].values)
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算RMSE  
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]));
    return dir_list

In [11]:
dev=100
lin=np.linspace(0.001,1,1000)
for alpha in lin:
    
    dict_W = dictW(alpha)
    
    dict_r = dictr(dict_W)

    dir_list=dirlist(dict_r)
    
    temp=np.mean(dir_list)
                      
    #误差判断
    if(temp>=dev):
        continue
    else:
        dev=temp
        al=alpha

alpha=al
dict_W = dictW(alpha)
dict_r= dictr(dict_W)

In [12]:
alpha

0.041

In [13]:
np.array(list(dict_r.values())).mean()

429.4291309882982

In [14]:
import pickle
with open('药物对.pkl', 'wb') as file:
    pickle.dump(dict_W, file)
    pickle.dump(dict_columns, file)
    pickle.dump(dict_high_lines, file) 